In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import os
import re
import torch
from torch.utils.data import Dataset, random_split, DataLoader
import numpy as np
import os

The problem ask to identify the spatial distribution of porosities given a certain density. This problem can be modeled as finding conditioned distribution p(x|c) where c is the density of the microstructure and x is the 30e3 vector of the porosities.

In [ ]:
path = '/gpfs/data/ssa/users/d602145/Workspace/scratch/Porosity/ETH/'
os.chdir(path)

Start with a standard data analysis before deciding which kind of strategy to apply to the problem


First We build a class to perform standard operations over the samples. Each sample is seen as a 3D distribution that we can plot in 3D. This is useful to understand the evolution of the spatial distribution of porosities. We want to check if some clusters appear increasing the level of porosity density. This imply that porosities are not issued from a random distribution, but there is a hidden criteria behind.

In [ ]:
from Lib.Data import PorosityDistribution



We build then 2 functions to extract datas. First one is used to create a dictionary of samples, where each sample is a microstructure. Second one create a dataframe with all the porosities of all the samples. This 2 will be used for different stats

In [ ]:
from Lib.Data import extract_microstructures,extract_porosities_points

In [ ]:
sample_path = os.getcwd()+'/Job_Assignment_Data/Job_Assignment_Data/'

In [ ]:
extracted_distributions = extract_microstructures(sample_path)
extracted_porosities, density_set = extract_porosities_points(sample_path)

In [ ]:
len(extracted_distributions.keys())
train_split = 0.8

train_keys = list(extracted_distributions.keys())[:int(train_split*len(extracted_distributions.keys()))]
test_keys = list(extracted_distributions.keys())[int(train_split*len(extracted_distributions.keys())):]

In [ ]:
train_keys = list(extracted_distributions.keys())

In [ ]:
print(len(train_keys),len(test_keys))

In [ ]:
def get_porosities_per_sample(keys,extracted_distributions):
    porosities = np.zeros((len(keys),extracted_distributions[0].distribution.shape[0]))
    densities = np.zeros((len(keys),1))
    for id,key in enumerate(keys):
        porosities[id,:] = extracted_distributions[key].as_dataframe()['porosity'].values
        densities[id,0] = extracted_distributions[key].density
        
    return porosities,densities

In [ ]:
porosities,densities = get_porosities_per_sample(train_keys,extracted_distributions)

In [ ]:
porosities.shape

In [ ]:
from sklearn.decomposition import PCA

pca_comp = 400

pca = PCA(n_components=400)

pca.fit(porosities)

In [ ]:
pca.explained_variance_ratio_.cumsum()[pca.explained_variance_ratio_.cumsum()<=0.99].shape

In [ ]:
px.scatter(pca.explained_variance_ratio_)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

pca_comp = 500

pca = PCA(n_components=pca_comp)
scaler_components = StandardScaler()
scaler_density = StandardScaler()

pc_components_porosities = pca.fit_transform(porosities)
scld_pc_components = scaler_components.fit_transform(pc_components_porosities)

data = pd.DataFrame(scld_pc_components, columns=[i for i in range(pca_comp)])

data['density'] = scaler_density.fit_transform(densities)

X, y =  data['density'].values.reshape(-1,1), data[0].values.reshape(-1,1),

In [ ]:
data.head()

In [ ]:
sns.heatmap(data.corr())

In [ ]:

g = sns.PairGrid(data.iloc[:200,::50], diag_sharey=False)
g.map_upper(sns.scatterplot, s=5)
g.map_lower(sns.kdeplot)
g.map_diag(sns.histplot,bins=30)

In [ ]:
g = sns.PairGrid(data.iloc[:200,-5::], diag_sharey=False)
g.map_upper(sns.scatterplot, s=5)
g.map_lower(sns.kdeplot)
g.map_diag(sns.histplot,bins=30)

In [ ]:
px.scatter(data,x='density',y=0)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures


poly_feat = PolynomialFeatures(3)

X = data['density'].values.reshape(-1,1)
y = data[0].values.reshape(-1,1)
X_poly = poly_feat.fit_transform(X)

deterministic_regressor = LinearRegression()
deterministic_regressor.fit(X_poly,y)

y_pred = deterministic_regressor.predict(X_poly)

data['0_rec'] = y_pred.reshape(-1)



In [ ]:
data.head()

In [ ]:
X.shape

In [ ]:

fig = px.scatter(x=X.reshape(-1), y=y.reshape(-1),
                 opacity=0.5,  # Set opacity for better visualization
                 title="Regression Plot: Actual vs. Predicted First Principal Component")

fig.add_scatter(x=X.reshape(-1), y=y_pred.reshape(-1),
                 opacity=0.5, mode='markers')  # Set opacity for better visualization)

fig.show()

In [ ]:
fig = px.scatter(data,x=0, y='0_rec',
                 opacity=0.5,  # Set opacity for better visualization
                 trendline="ols",  # Add trendline using Ordinary Least Squares
                 title="Regression Plot: Actual vs. Predicted Density")

# Add ideal line (y = x)
x_range = np.linspace(data[0].min(), data['0_rec'].max(), 100)
fig.add_scatter(x=x_range, y=x_range, mode='lines',
                line=dict(color='red', dash='dash'),
                name='Ideal Line (y = x)')

fig.update_layout(
    xaxis_title="Actual Density",
    yaxis_title="Predicted Density"
)

fig.show()

In [ ]:
stochastic_dimensions = [i for i in range(1,pca_comp)]
stochastic_dimensions.append('density')
stochastic_df = data[stochastic_dimensions].copy()



In [ ]:
stochastic_df.head()

In [ ]:
density_bins=30
stochastic_df['density_bins'] = pd.cut(stochastic_df['density'],bins=density_bins)

means = stochastic_df.groupby(['density_bins']).mean()
stds = stochastic_df.groupby(['density_bins']).std()

In [ ]:
means.head()

In [ ]:
dim_means = means.iloc[:,:-1]
dim_stds = stds.iloc[:,:-1]

densities = np.zeros_like(dim_stds.values)
dim_coord = np.zeros_like(densities)

for i in range(densities.shape[0]):
    densities[i,:] = means['density'].iloc[i]
    dim_coord[i,:] = dim_stds.columns.values
    
densities = pd.DataFrame(densities,columns=dim_means.columns,index=dim_means.index)
dim_coord = pd.DataFrame(dim_coord,columns=dim_means.columns,index=dim_means.index)

In [ ]:
dim_means.head()

In [ ]:
dim_stds.head()

In [ ]:
densities.head()

In [ ]:
dim_coord.head()

In [ ]:
X = np.zeros((densities.shape[0],densities.shape[1],4))
X[:,:,0] = dim_means.values
X[:,:,1] = dim_stds.values
X[:,:,2] = densities.values
X[:,:,3] = dim_coord.values

gaussian_parameters = pd.DataFrame(X.reshape(densities.shape[0]*densities.shape[1],4),columns=['dim_mean','dim_std','density','dim'])

In [ ]:
gaussian_parameters.head()

In [ ]:
px.scatter_3d(gaussian_parameters,x='density',y='dim',z='dim_std',color='dim_std')

In [ ]:
px.scatter_3d(gaussian_parameters,x='density',y='dim',z='dim_mean',color='dim_mean')

In [ ]:
gaussian_parameters.head()

In [ ]:
px.scatter_3d(gaussian_parameters,x='density',y='dim',z='dim_std',color='dim_std')

In [ ]:
px.scatter_3d(gaussian_parameters,x='density',y='dim',z='dim_mean',color='dim_mean')

In [ ]:
X_test = np.random.rand(500,2)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

X = gaussian_parameters[['density','dim']].values
y = gaussian_parameters[['dim_mean','dim_std']].values

statistics_regressor = KNeighborsRegressor(n_neighbors=2)

statistics_regressor.fit(X,y)

y_pred = statistics_regressor.predict(X)

In [ ]:
gaussian_parameters['rec_mean'] = y_pred[:,0]
gaussian_parameters['rec_std'] = y_pred[:,1]

In [ ]:
px.scatter_3d(gaussian_parameters,x='density',y='dim',z='rec_std',color='rec_std')

In [ ]:
px.scatter_3d(gaussian_parameters,x='density',y='dim',z='rec_mean',color='rec_mean')

In [ ]:
X_dim_test = np.random.randint(1,199,size=1000)
X_density_test = np.random.rand(1000)
X_test = np.zeros((1000,2))
res = np.zeros((1000,4))

X_test[:,0] = X_density_test
X_test[:,1] = X_dim_test
res[:,0] = X_density_test
res[:,1] = X_dim_test

In [ ]:
y_test = statistics_regressor.predict(X_test)
res[:,2:] = y_test

In [ ]:
res = pd.DataFrame(res, columns=['density','dim','rec_mean','rec_std'])



In [ ]:
px.scatter_3d(gaussian_parameters,x='density',y='dim',z='rec_mean',color='rec_mean')

In [ ]:
px.scatter_3d(gaussian_parameters,x='density',y='dim',z='rec_std',color='rec_std')

In [ ]:
import numpy as np

def generate(samples, density):

    components = np.zeros((samples,pca_comp))
    sc_density = scaler_density.transform(np.array([[density]]))

    principal = deterministic_regressor.predict(poly_feat.transform(sc_density))
    components[:,0]=principal
    sc_density = sc_density[0,0]

    for dimension in range(1,pca_comp):
        
        X = np.array([[sc_density,dimension]])
        
        statistics = statistics_regressor.predict(X)
        #print(statistics,dimension)
        mean_dimension = statistics[0,0]
        std_dimensions = statistics[0,1]
        
        components[:,dimension] = np.random.normal(loc = mean_dimension, scale = std_dimensions,size=samples)
        
    porosities = pca.inverse_transform(scaler_components.inverse_transform(components))

    #print(porosities.shape)
        
    return porosities
    

In [ ]:
porosities = generate(100,0.2)

In [ ]:
porosities

In [ ]:
density = 0.5
porosities = generate(100,density)


template = np.load(os.path.join(sample_path, 'distribution_000_0.821.npy'))

array_data = template
array_data[:,3] = porosities[0,:]
# Store in the dictionary
sample = PorosityDistribution(array_data, density=density)
sample.plot_porosity_distribution(porosity=0.5)